In [76]:
######## Importing the necessary libraries #########http://localhost:8888/notebooks/rfi_RRI/RFI_Heat_MAP_new_master_cluster.ipynb#


import pandas as pd
import numpy as np
import healpy as hp
import matplotlib.pyplot as plt
from astropy.constants import c, k_B,R_earth 
import h5py





In [77]:
##Logging calls###

import logging
logname = "power_cube.log"
logging.basicConfig(
    filename=logname,
    filemode="a",
    format="%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s",
    datefmt="%H:%M:%S",
    level=logging.DEBUG,
)

In [78]:
### Constants and file path###


fstart=87 # in MHz
fstop= 108 # in MHz
fres= 0.5# in MHz
freq=np.arange(fstart,fstop,fres)
logging.info("User defined Frequency axis generated")
nside = 4
min_alt=400  #in km
max_alt=36000 #in km
data_point=1
altitude= np.logspace(np.log10(min_alt),np.log10(max_alt),data_point) 
logging.info("Altitudes generated")
R_E=R_earth.to('km') # Radius of Earth in km


file_path="/home/ghoshsonia/rfi_RRI/Final_Revised.csv"
#output_file_path

In [79]:
##### Fetching the cleaned FM transmitter data for the countries: CANADA, AUSTRALIA ,GERMANY,USA and SOUTH AFRICA ####
#####  EIRP in Watts #####
##### Latitude ranges from 90(N.Pole) to 0 (EQUATOR) to -90(S.Pole) ########
##### Longitude ranges between 0 and 360 degree eastwards ############


#Reading the CSV file
df=pd.read_csv("/home/ghoshsonia/rfi_RRI/Final_Revised.csv")

#Removing the Null/missing values in the CSV file
df.dropna(subset = ["Latitude in degrees"], inplace=True)


In [80]:
freq=np.arange(fstart,fstop,fres)

#### Array consisting of frequencies from the dataset
data_freq=df['Frequency(MHz)'].values



diff=np.zeros((len(data_freq),len(freq)),dtype=object)
freq_arr=np.zeros(len(data_freq))


for i in range(len(data_freq)):
    for j in range(len(freq)):
        diff[i][j]=abs(data_freq[i]-freq[j])
        freq_arr[i]=freq[np.argmin(diff[i])]

In [81]:
df['New Frequency']=freq_arr


In [82]:
#Resolution of the map
logging.info("-----------Resolution of the map---------")
logging.info("The number of pixels for the given NSIDE: " + str(hp.nside2npix(nside)))
logging.info("Approximate resolution in degrees for given nside: " + str(np.degrees(hp.nside2resol(nside))))




In [83]:

######-------------------Allocating Pixel number to the Latitude and Longitude of each Tx in the CSV---------#######



# Healpy pixel number when input angles are assumed to be longitude and latitude in degrees
pixel_indices = hp.ang2pix(nside, df['Longitude in degrees'].to_numpy() ,df['Latitude in degrees'].to_numpy(),lonlat=True)
df['Pixel_number']=pixel_indices 
logging.info("The pixel numbers corresponding to the Latitude and Longitude of each Tx is generated")


/home/ghoshsonia/.local/lib/python3.8/site-packages/healpy/pixelfunc.py:482: RuntimeWarning: invalid value encountered in _ang2pix_ring
  return pixlib._ang2pix_ring(nside, theta, phi)


In [84]:
#######---------------Conversion of the pixel numbers w.r.t the given NSIDE to corresponding angular coordinates--------#######


NPIX = hp.nside2npix(nside) # Storing the number of pixels of the map corresponding to the given NSIDE
arr=np.arange(NPIX) #Create an an array of pixel numbers with respect to the NSIDE

phi,theta = (hp.pix2ang(nside, ipix=arr,lonlat=True)) # Array of the angular coordinates co-latitude(theta) 
                                                        #and longitude(phi) in degrees 
                                                       # With respect to the given NSIDE
logging.info("The number of pixels" + str(NPIX))


In [85]:
#Calculation of Elevation angle

x=np.zeros((NPIX,NPIX))
y=np.zeros((NPIX,NPIX))
elev_ang=np.zeros((len(altitude),NPIX,NPIX))
#R_E=6400
for k in range(len(altitude)):
    for i in range(len(theta)):
        for j in range(len(theta)):
        
            x[i,j]=((np.cos(np.radians(theta[i])))*(np.cos(np.radians(theta[j])))*(np.cos(np.radians(phi[j]-phi[i])))+(np.sin(np.radians(theta[i])))*(np.sin(np.radians(theta[j]))))
            y[i,j]=(np.arccos(x[i,j]))
            B=(altitude[k] +R_E.value)/R_E.value
            elev_ang[k,i,j]=-(np.degrees(np.arctan((B-np.cos(np.radians(y[i,j])))/np.sin(np.radians(y[i,j]))))-y[i,j])
            
#logging.info("Calculation of elevation angle finished")

<ipython-input-85-28634b723fdc>:14: RuntimeWarning: divide by zero encountered in double_scalars
  elev_ang[k,i,j]=-(np.degrees(np.arctan((B-np.cos(np.radians(y[i,j])))/np.sin(np.radians(y[i,j]))))-y[i,j])


In [86]:

# Function for Radiation pattern of the satellite antenna beam #
beam_pattern=np.zeros((len(altitude),NPIX,NPIX))
beam = lambda theta, phi: (np.cos(np.radians(theta)))**2  


In [87]:
# Assuming the satellite antenna beam to be symmetric across azimuth #
# Calculation of the beam pattern #

az=0
beam_pattern = beam(elev_ang,az)
logging.info("Beam pattern cube generated")


In [88]:
###########----------------- Calculation of Field of view of the satellite---------#######
###########-----------------Considering Nadir-pointing Field of View Geometry-------######
##########-----------------Considering the FOV of the satellite to be tangent to the surface of the Earth------######


FOV=np.zeros(len(altitude))
for i in range(0,len(altitude)):
# Consider a case of full coverage under elevation of 0 º
    #Rad= 6371 # Mean radius of Earth in km
    
    FOV[i]= 2*np.arcsin(R_E.value/(R_E.value+ altitude[i]))  # Field of view for maximal coverage in radians when elevation is 0 º 
    
    logging.info(f"The Field of view of the satellite at a height of {altitude[i]:.2f} km is {FOV[i]:.2f} radians")
    

In [89]:
###########----------------- Calculation of the radius of the FOV of the satellite---------#######



# The surface of the coverage area of the Earth depends on the central angle
Central_angle=np.zeros(len(altitude))
for i in range(0,len(altitude)):
    Central_angle[i]=np.arccos(R_E.value/(R_E.value+altitude[i])) # Central angle in radians
    
    Dia_of_FOV=2*Central_angle*R_E.value  # Diameter of the FOV (disc on the Earth's surface)in km
    Rad_of_FOV= Dia_of_FOV/2 # Radius of the FOV in km
    Rad_of_FOV=Rad_of_FOV/R_E.value  # Radius of the FOV in Radians
    logging.info(f"The Radius of the Field of View for a height of {altitude[i]:.2f} km in radians is {Rad_of_FOV[i]:.2f}")


In [90]:
###############-----Storing indices of the pixel number that are inside the circle/disc(FOV) wrt the altitude-----------######


vec1 = hp.ang2vec(phi,theta,lonlat=True) #Using ang2vec convert angles that is co-latitude and longitude in radians
                                        #to 3D position vector
    
disc=np.zeros((len(arr),len(Rad_of_FOV)),dtype=object)#Array of indices of the pixel number that are inside the 
                                                      #circle/disc specified by vec and radius
def pixel_disc(a,b):
    for i in range(len(a)):
        for j in range(len(b)):
            disc[j][i]=hp.query_disc(nside, vec1[j], radius=a[i])
pixel_disc(Rad_of_FOV,arr)

In [91]:
######-----Storing indices of the pixel number that are common between the FOV disc and the satellite pixel no------####


Comm_pix=np.zeros((len(disc[:,i]),len(Rad_of_FOV)),dtype=object)

for i in range(len(Rad_of_FOV)):
    for k in range(len(disc[:,i])):
     

         Comm_pix[k][i]=np.intersect1d(pixel_indices,disc[k][i])#Array of indices of the pixel number that are
                                                           #common between the FOV disc and the satellite pixel no
         

In [92]:
######-----Storing indices of the pixel number that are common between the FOV disc and the satellite pixel no ------#
######-----And the same pixel number of the transmitters-----##



######--- Initializing the array to store the pixel number that are common between the FOV disc ---####
######                        and the satellite pixel no    ######
Comm_TX=np.zeros((len(Comm_pix),len(Rad_of_FOV)),dtype=object)
found_common=np.zeros(( len(Comm_pix),len(Rad_of_FOV)),dtype=object)

for j in range(len(Rad_of_FOV)):
    for i in range(len(Comm_pix)):
        
        Comm_TX[i][j]=set(Comm_pix[i][j])
        found_common[i][j] = [l for l in pixel_indices if l in Comm_TX[i][j]]#Array of indices of the pixel number that 
                                            #are common between the FOV disc and the satellite pixel no with
                                                    # Tx having same pixel number
                                                                       
logging.info("Common Pixel matrix generated")         

In [93]:
#######-------Store the values of the received power in Watt,dBm and Kelvin wrt altitude----#####
######------- Calculation of the received power using Friis Transmission Equation--------#######
######-------Considering isotropic transmitter and receiver with gain =1 -------------#########





#res=fres*1e6 #Bandwidth
res=200*1e3
Rx_Power= np.zeros((len(df),len(altitude)))
Rx_Power_in_Kelvin=np.zeros((len(df),len(altitude)))
for i in range(0,len(altitude)):
    for j in range(0,len(df)):
        wavelength= c.value/(df.iloc[j]['Frequency(MHz)']*1e6)
        Rx_Power[j][i]= ((df.iloc[j]['EIRP'])*(wavelength)**2)/(4*np.pi*altitude[i]*1e3)**2 #the Friis Transmission Equation
        
        Rx_Power_in_Kelvin[j][i]=Rx_Power[j][i]/(k_B.value*res)#in Kelvin
        

       

In [94]:
##### ---Create dataframe to store the values of the received power in Kelvin w.r.t the altitude----#####

##### The first three columns of the dataframe indicate the altitude [0=400 km, 1= 3794 km , 2= 36000 km ] #####
##### The column will vary with the user input for the number of altitudes ######

df_data=pd.DataFrame(Rx_Power_in_Kelvin)
df_data['Pixel_number']=pixel_indices
df_data['New Frequency']=freq_arr

logging.info("Dataframe to store the values of the received power in Kelvin generated ")   

In [95]:
#########  Create dataframes for the FOV for each pixel position of the satellite at different altitudes #######



######--- Initializing the array to store the dataframes for the FOV for each pixel position of the satellite #####
######            at different altitudes   ######


df_1=np.zeros((len(found_common),len(Rad_of_FOV)),dtype=object)

for j in range(len(Rad_of_FOV)):
    for i in range(len(found_common)):
        df_1[i][j]= df_data.loc[df_data['Pixel_number'].isin(found_common[i][j])]
        df_1[i][j][j]=df_1[i][j][j]*beam_pattern[j,i,found_common[i,j]]
       # df_1[i][j]['Rx_Power in Watt at 3795 km']=df_1[i][j]['Rx_Power in Kelvin at 3795 km']*beam_pattern[i,found_common[i,j]]
       # df_1[i][j]['Rx_Power in Watt at 36000 km']=df_1[i][j]['Rx_Power in Kelvin at 36000 km']*beam_pattern[i,found_common[i,j]]
        df_1[i][j]= df_1[i][j].groupby(['New Frequency']).sum()  # df9['Pixel_number']= Column consisting of 
        df_1[i][j]= df_1[i][j].reset_index() 

<ipython-input-95-674896c093ef>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1[i][j][j]=df_1[i][j][j]*beam_pattern[j,i,found_common[i,j]]


In [96]:
## Initializing the 3D array to store the received power for each pixel, each frequency and at different altitudes ###

power_output=np.zeros((len(Rad_of_FOV),NPIX,len(freq)))#,dtype=object)


In [97]:
## Create the 3D array to store the received power for each pixel, each frequency and at different altitudes ##
## k: loops along the length of the number of altitude
## m: loops along the length of the number of pixels based on the given NSIDE
## l: loops along the length of the number of frequencies in the frequency axis defined by user



for k in range(len(Rad_of_FOV)):
    for m in range(NPIX):    
        for l in range(len(freq)):
            if (df_1[m][k][k][df_1[m][k]['New Frequency'] == freq[l]].values).size==0:# checking for empty dataframes
                power_output[k][m][l]=0
            else: 
                power_output[k][m][l]=float(df_1[m][k][k][df_1[m][k]['New Frequency'] == freq[l]].values)
logging.info("3D Power cube generated")      

In [98]:
test = h5py.File("testfile.hdf5", "w")
dset1 = test.create_dataset("Received_power", data=power_output)
dset2 = test.create_dataset("Frequency", data=freq)
dset3 = test.create_dataset("Altitude", data=altitude)
#dset4  = test.create_dataset("NSIDE", data=nside)
dset5 = test.create_dataset("History", data='This is just a test with hard coded values')
test.close()

In [99]:
test_plot00 = np.zeros(hp.nside2npix(nside))
test_plot00[0:np.size(power_output[0,:,6])] = power_output[0,:,6]
hp.mollview(test_plot00,cmap='inferno',unit="in Kelvin",title="Mollweide projection at an altitude 400 km for Received power in Kelvin at 90 MHz",flip='geo',norm='hist')#,cb_orientation="vertical",title="Mollweide projection for Received power in Kelvin at 88.9MHz",flip='g')
hp.graticule()
plt.savefig('Map_400_at_90.png')
plt.close()
logging.info("Plot for alt 400 at 90 generated ")

In [100]:
test_plot01 = np.zeros(hp.nside2npix(nside))
test_plot01[0:np.size(power_output[len(altitude)-1,:,6])] = power_output[len(altitude)-1,:,6]
hp.mollview(test_plot01,cmap='inferno',unit="in Kelvin",title="Mollweide projection at highest altitude in km for Received power in Kelvin at 90 MHz",flip='geo',norm='hist')#,cb_orientation="vertical",title="Mollweide projection for Received power in Kelvin at 88.9MHz",flip='g')
hp.graticule()
plt.savefig('Map_highest_alt_at_90.png')
plt.close()
logging.info("Plot for highest alt at 90 generated ")

In [101]:
logging.info("Finished Run")
logging.shutdown()